## This notebook shows how to use Orchestrator APIs for user experiments

In [7]:
import os
from fabric.cli.credential import CredMgr
from fabric.cli.orchestrator import Orchestrator
import json

### Fabric Tokens
Fabric has 2 kinds of tokens:<br>
- Identity : required for Control/Measurement Framework APIs. Identity Token is valid upto an hour.
- Refresh : required to generate new Identity Tokens valid. Refresh Token is valid for 24 hours.

Fabric Identity token is required for Control/Measurement Framework APIs.<br>

When user logins to Jupyterhub after authenticating against CILogon, OIDC refresh token is derived. <br>
This token is available as the environment variable `CILOGON_REFRESH_TOKEN`.<br>

On the first login, we use `CILOGON_REFRESH_TOKEN` to generate new Fabric Identity Token and Fabric Refresh Token.<br>
For any subsequent use, we use Fabric Refresh Token. On every refresh, Fabric Refresh Token is changed and updated.<br>

NOTE: These steps are required for any experiments on Fabric Testbed.<br>

In [8]:
try:
   %store -r fabric_refresh_token
except NameError as e:
    pass
if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

Fabric Refresh Token https://cilogon.org/oauth2/refreshToken/192d627dd8c0c49584098731f7f4e40b/1604664454596
CILOGON_REFRESH_TOKEN environment variable: https://cilogon.org/oauth2/refreshToken/5c48c3839e258582da43a0d430af1f1f/1604628183530


### Get new Fabric Identity Token and update Fabric Refresh Token

In [9]:
try:
    refresh_res = CredMgr.refresh_token(project_name='all', scope='all', refresh_token=fabric_refresh_token)
    print("New Tokens received: {}".format(json.dumps(refresh_res)))
    fabric_id_token=refresh_res['id_token']
    fabric_refresh_token=refresh_res['refresh_token']
    print()
    print("New Refresh Token: {}".format(fabric_refresh_token))
    print()
    print("Stored new Refresh Token")
    %store fabric_refresh_token
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

New Tokens received: {"id_token": "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJlbWFpbCI6Imt0aGFyZTEwQGVtYWlsLnVuYy5lZHUiLCJnaXZlbl9uYW1lIjoiS29tYWwiLCJmYW1pbHlfbmFtZSI6IlRoYXJlamEiLCJuYW1lIjoiS29tYWwgVGhhcmVqYSIsImlzcyI6Imh0dHBzOi8vY2lsb2dvbi5vcmciLCJzdWIiOiJodHRwOi8vY2lsb2dvbi5vcmcvc2VydmVyQS91c2Vycy8xMTkwNDEwMSIsImF1ZCI6ImNpbG9nb246L2NsaWVudF9pZC8xMjUzZGVmYzYwYTMyM2ZjYWEzYjQ0OTMyNjQ3NjA5OSIsInRva2VuX2lkIjoiaHR0cHM6Ly9jaWxvZ29uLm9yZy9vYXV0aDIvaWRUb2tlbi8zNTU5YmMyNDg0ZDc2MGUyOGM4MWZjYzNlOTE5ZTA3NS8xNjA0NjY0MzM0MzA4IiwiYXV0aF90aW1lIjoiMTYwNDY2NDMzNCIsImV4cCI6MTYwNDY2ODA4MSwiaWF0IjoxNjA0NjY0NDgxLCJyb2xlcyI6WyJDTzptZW1iZXJzOmFjdGl2ZSIsIkNPOkNPVTpKdXB5dGVyaHViOm1lbWJlcnM6YWN0aXZlIiwiQ086Q09VOnByb2plY3QtbGVhZHM6bWVtYmVyczphY3RpdmUiXSwic2NvcGUiOiJhbGwiLCJwcm9qZWN0IjoiYWxsIn0.Gp9PmHT3-qGBWoIjT2lSQmRcSsgs41Es9HwKN_g99ZayfXbYWXIgDlcBR2C1e90J_1RE-sIobZ69NOeIvq7olatjBi2cLFDHlUucZJscQAzA1lkgia9XTdqAxBBFAjC6G4jCYJ1GaJM6xrxbHKQ8UNvEM2TzW5VpKQCf5W5vMt7lEXz-IeQ_BREBm1Fpq-6nVK-wrxs-aA8W-P4cgKwJjfBXa0aA4D-w

### Orchestrator API example to query for available resources

In [11]:
resources = Orchestrator.resources(id_token=fabric_id_token)

print(resources)

{'value': '<?xml version="1.0" encoding="UTF-8"?><graphml xmlns="http://graphml.graphdrawing.org/xmlns" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd"><key id="GraphID" for="node" attr.name="GraphID" attr.type="string"/><key id="labels" for="node" attr.name="labels" attr.type="string"/>\n<key id="Labels" for="node" attr.name="Labels" attr.type="string"/><key id="label" for="node" attr.name="label" attr.type="string"/><key id="Capacities" for="node" attr.name="Capacities" attr.type="string"/><key id="Layer" for="node" attr.name="Layer" attr.type="string"/><key id="labels" for="node" attr.name="labels" attr.type="string"/><key id="Name" for="node" attr.name="Name" attr.type="string"/><key id="LabelDelegations" for="node" attr.name="LabelDelegations" attr.type="string"/><key id="Type" for="node" attr.name="Type" attr.type="string"/><key id="ADMGraphIDs" for="node" attr.n